In [ ]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
from eval_reallabor.reallabor_metrics import MetricsManager
import data_utils

def create_ensemble_prediction_eval_df(eval_df: pd.DataFrame) -> pd.DataFrame:

    eval_df = eval_df.copy()
    ensemble_defining_cols = ['model_id', 'feature', 'steps', 'sample', 'prewarm_steps', 'test_day']
    agg_funcs = {col: 'mean' if pd.api.types.is_numeric_dtype(eval_df[col]) else 'first' for col in eval_df.columns}
    ensemble_eval_df = eval_df.groupby(ensemble_defining_cols, as_index=False, dropna=False, sort=False).agg(agg_funcs)   # CAUTION: sort_result=True results in buggy change prediction
    ensemble_eval_df = ensemble_eval_df.iloc[np.random.permutation(len(ensemble_eval_df))]
    return ensemble_eval_df


path = data_utils.join_ordinal_bptt_path('results/v3_MRT2_every_day/00_summary_7stepsahead_interv/evaluation.csv')
df = pd.read_csv(path)
csc = []
for i in range(100):
    edf = create_ensemble_prediction_eval_df(df)
    M = MetricsManager(edf, ['latent_model'])
    csc.append(M.change_sign_correct())
cols = ['model_id', 'feature', 'steps', 'sample', 'prewarm_steps', 'test_day']

In [45]:
M.change_sign_correct()

latent_model
clipped-shallow-PLRNN    0.810328
Name: change sign correct, dtype: float64

In [21]:
edf.groupby(['model_id', 'feature'])['steps'].is_monotonic_increasing.all()

np.True_

In [26]:
nedf = edf.set_index('feature', append=True)['ground_truth'].unstack('feature')

In [22]:
edf[cols].head(20)

,model_id,feature,steps,sample,prewarm_steps,test_day
0,data_12600_101.csv_participant_101_date_104.0,EMA_concentration,0.0,0.0,0.0,104.0
1,data_12600_101.csv_participant_101_date_104.0,EMA_concentration,1.0,0.0,0.0,104.0
2,data_12600_101.csv_participant_101_date_104.0,EMA_concentration,2.0,0.0,0.0,104.0
3,data_12600_101.csv_participant_101_date_104.0,EMA_concentration,3.0,0.0,0.0,104.0
4,data_12600_101.csv_participant_101_date_104.0,EMA_concentration,4.0,0.0,0.0,104.0
5,data_12600_101.csv_participant_101_date_104.0,EMA_concentration,5.0,0.0,0.0,104.0
6,data_12600_101.csv_participant_101_date_104.0,EMA_concentration,6.0,0.0,0.0,104.0
7,data_12600_101.csv_participant_101_date_104.0,EMA_concentration,7.0,0.0,0.0,104.0
8,data_12600_101.csv_participant_101_date_104.0,EMA_confidence,0.0,0.0,0.0,104.0
9,data_12600_101.csv_participant_101_date_104.0,EMA_confidence,1.0,0.0,0.0,104.0


In [29]:
var_path = data_utils.join_ordinal_bptt_path('results/v3_MRT2_SimpleModels_every_day/00_summary_7stepsahead_interv/evaluation.csv')
var_df = pd.read_csv(path)
var_df = var_df[var_df['latent_model']=='VAR1']
var_edf = create_ensemble_prediction_eval_df(var_df, sort_result=True)